# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [34]:
#Import CSV
weather = pd.read_csv("../WeatherPy/Outputs/Weather_Data")
weather

,City,County,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Bluff,NZ,-46.6000,168.3333,12.24,92,100,12.91
1,Ushuaia,AR,-54.8000,-68.3000,20.00,52,0,3.68
2,Tumalim,PH,14.0801,120.7231,25.69,89,75,1.54
3,Rikitea,PF,-23.1203,-134.9692,25.60,75,50,4.06
4,Lashma,RU,54.9321,41.1438,-13.63,94,100,3.11
...,...,...,...,...,...,...,...,...
559,Havre-St-Pierre,CA,50.2334,-63.5986,-17.00,55,1,6.69
560,Lipari,IT,38.4674,14.9540,20.56,44,66,0.45
561,Sárvár,HU,47.2539,16.9352,2.21,100,90,2.06
562,Abu Samrah,SY,35.3029,37.1841,16.13,52,0,6.84


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [35]:
# Store Latitude and Longitude into  locations and make float
locations = weather[["Latitude", "Longitude"]].astype(float)

#Ensure humidity is float typle
humid = weather["Humidity"].astype(float)

In [59]:
# Create a humidity Heatmap layer
fig = gmaps.figure(center=(30, 0), zoom_level=2)

humid_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(humid_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
#Slim down df to locations with no cloudiness, wind < 10 mph, and temps between 21-24 degrees C (70-75 degree F)
#No cloudiness
no_clouds = weather.loc[weather['Cloudiness'] == 0]
no_clouds

#winds less than 10 mph
slow_wind = no_clouds.loc[no_clouds['Wind Speed'] < 10]
slow_wind

#temp between 18-26
ideal_weather = slow_wind.loc[(slow_wind['Temperature'] > 21) & (slow_wind['Temperature'] < 24)]
print(f'There are {len(ideal_weather)} cities in this dataset')
ideal_weather

There are 7 cities in this dataset


,City,County,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
131,Lebu,CL,-37.6167,-73.6500,21.10,60,0,2.16
161,Abu Dhabi,AE,24.4667,54.3667,23.57,73,0,5.14
289,Najrān,SA,17.4924,44.1277,22.00,40,0,4.12
335,Ajdabiya,LY,30.7554,20.2263,23.50,20,0,2.32
336,Marsh Harbour,BS,26.5412,-77.0636,23.79,74,0,3.71
397,Dakar,SN,14.6937,-17.4441,22.00,64,0,6.69
513,Walvis Bay,NaN,-22.9575,14.5053,22.64,64,0,6.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# geocoordinates
target_coordinates = "43.6187102, -116.2146068"
target_search = "Chinese"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# Get hotel data from Google Places API
hotal_df = c.acs5.get(("B01003_001E", "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Poverty Rate"]]

# Visualize
print(len(census_pd))
census_pd.head()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
